In [1]:
from db1_preprocess_utils import *
from feature_extraction import *
from experiment_one_utils import *

In [2]:
from tensorflow.keras import layers
from tensorflow import keras

import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
from sklearn.model_selection import KFold
from scipy.signal import stft

In [3]:
path = 'D:\\AI-Workspace\\sEMGClassification\\GestureClassificationUsingCViT\\data\\DB1\\raw'

signal_type='raw'
input_type='stft'
n_channels=64
low_cut=20
high_cut=500
order=6 
window_size=50
overlap=50
fs=2048

whiten=False
center=False
extend=False
extend_size=0
normalize=False
mu=0

start_subject = 1
total_subject = 20
session=[1,2]
ratio = 0.2

noise_db = [5,10,15]
std = 1
stochastic_depth_rate = 0.1

In [4]:
data, label, X_test, y_test = get_experiment_data(path, subjects=start_subject, sessions=session,
                                                       signal_type='raw', input_type=input_type, 
                                                       channels=n_channels, low_cut=20, high_cut=500,
                                                      order=6, window_size=window_size, overlap=overlap, fs=fs, 
                                                      extend=extend, center=center, 
                                                      extend_size=extend_size, whiten=whiten,
                                                      normalize=False, mu=0, ratio=0.2)

####################################################################################
Loading subject 1
####################################################################################
Size of the input data is (17152, 64, 52, 3)
The input label shape is (17152,)
The total number of classes is 34
************************************************************************************
Loaded STFT input data
************************************************************************************


In [5]:
#Hyper-parameter

input_shape = data.shape[1:]
patch_size = 8
n_patches = 12 
dims = n_patches + 1 
n_transformer_layers = 2
n_heads = 2
transformer_units = [dims * 2,dims]  
mlp_head_units = [128, 34]
n_classes = len(np.unique(label))
stochastic_depth = False

stochastic_depth_rate = 0.1

n_batches = 32
n_epochs = 150
n_folds = 3
type_of_experiment = 1

In [6]:
model = vit_gesture_classification(input_shape, dims, patch_size, n_patches,
                                  n_transformer_layers, n_heads, transformer_units,
                                   mlp_head_units, n_classes,stochastic_depth,stochastic_depth_rate)

model.summary()

Model: "sEMG-Decomposition"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 64, 52, 3)]  0           []                               
                                                                                                  
 covolution_layer (Sequential)  (None, 32, 26, 13)   21121       ['input_1[0][0]']                
                                                                                                  
 generate__patches (Generate_Pa  (None, None, 832)   0           ['covolution_layer[0][0]']       
 tches)                                                                                           
                                                                                                  
 embed__position (Embed_Positio  (None, 12, 13)      10985       ['generate__patc

In [7]:
run_experiment(n_folds, n_batches, n_epochs, start_subject, total_subject, session, path, 
                       input_type,n_channels, window_size,extend, extend_size, center, whiten, ratio, noise_db, std,
                      input_shape, dims, patch_size, n_patches,
                       n_transformer_layers, n_heads, transformer_units,
                       mlp_head_units, n_classes,stochastic_depth,stochastic_depth_rate, type_of_experiment)

************************************************
Experiment One
************************************************
####################################################################################
Loading subject 1
####################################################################################
Size of the input data is (17152, 64, 52, 3)
The input label shape is (17152,)
The total number of classes is 34
************************************************************************************
Loaded STFT input data
************************************************************************************
Adding noise to STFT input test data
Adding white noise to STFT input test data
---------------------------------------------------
Training for fold 1 -------
Epoch 1/150
358/358 [==============================] - 34s 59ms/step - loss: 3.2908 - accuracy: 0.0860
Epoch 2/150
358/358 [==============================] - 21s 60ms/step - loss: 2.4103 - accuracy: 0.2689
Epoch 3/150
358/358 [======

358/358 [==============================] - 20s 56ms/step - loss: 0.2506 - accuracy: 0.9235
Epoch 73/150
358/358 [==============================] - 20s 56ms/step - loss: 0.2506 - accuracy: 0.9243
Epoch 74/150
358/358 [==============================] - 20s 56ms/step - loss: 0.2554 - accuracy: 0.9228
Epoch 75/150
358/358 [==============================] - 20s 56ms/step - loss: 0.2434 - accuracy: 0.9267
Epoch 76/150
358/358 [==============================] - 20s 56ms/step - loss: 0.2510 - accuracy: 0.9218
Epoch 77/150
358/358 [==============================] - 20s 56ms/step - loss: 0.2435 - accuracy: 0.9254
Epoch 78/150
358/358 [==============================] - 20s 56ms/step - loss: 0.2459 - accuracy: 0.9229
Epoch 79/150
358/358 [==============================] - 20s 56ms/step - loss: 0.2435 - accuracy: 0.9245
Epoch 80/150
358/358 [==============================] - 20s 56ms/step - loss: 0.2399 - accuracy: 0.9241
Score for fold  1: loss of 0.13927999138832092; accuracy of 96.2574303150177%

358/358 [==============================] - 43s 119ms/step - loss: 0.7534 - accuracy: 0.7622
Epoch 15/150
358/358 [==============================] - 43s 119ms/step - loss: 0.7129 - accuracy: 0.7721
Epoch 16/150
358/358 [==============================] - 43s 119ms/step - loss: 0.6872 - accuracy: 0.7857
Epoch 17/150
358/358 [==============================] - 43s 119ms/step - loss: 0.6493 - accuracy: 0.8003
Epoch 18/150
358/358 [==============================] - 43s 119ms/step - loss: 0.6090 - accuracy: 0.8104
Epoch 19/150
358/358 [==============================] - 43s 119ms/step - loss: 0.5934 - accuracy: 0.8127
Epoch 20/150
358/358 [==============================] - 43s 119ms/step - loss: 0.5532 - accuracy: 0.8307
Epoch 21/150
358/358 [==============================] - 43s 119ms/step - loss: 0.5400 - accuracy: 0.8323
Epoch 22/150
358/358 [==============================] - 43s 119ms/step - loss: 0.5262 - accuracy: 0.8342
Epoch 23/150
358/358 [==============================] - 43s 119ms/st

360/360 [==============================] - 22s 61ms/step - loss: 2.5079 - accuracy: 0.2362
Epoch 4/150
360/360 [==============================] - 22s 61ms/step - loss: 2.1527 - accuracy: 0.3166
Epoch 5/150
360/360 [==============================] - 22s 61ms/step - loss: 1.8958 - accuracy: 0.3861
Epoch 6/150
360/360 [==============================] - 22s 61ms/step - loss: 1.6791 - accuracy: 0.4546
Epoch 7/150
360/360 [==============================] - 22s 62ms/step - loss: 1.5198 - accuracy: 0.5091
Epoch 8/150
360/360 [==============================] - 22s 61ms/step - loss: 1.3901 - accuracy: 0.5484
Epoch 9/150
360/360 [==============================] - 22s 61ms/step - loss: 1.2839 - accuracy: 0.5836
Epoch 10/150
360/360 [==============================] - 22s 61ms/step - loss: 1.1922 - accuracy: 0.6088
Epoch 11/150
360/360 [==============================] - 22s 61ms/step - loss: 1.1172 - accuracy: 0.6410
Epoch 12/150
360/360 [==============================] - 22s 61ms/step - loss: 1.055

Epoch 82/150
360/360 [==============================] - 22s 61ms/step - loss: 0.3744 - accuracy: 0.8805
Epoch 83/150
360/360 [==============================] - 22s 61ms/step - loss: 0.3712 - accuracy: 0.8841
Epoch 84/150
360/360 [==============================] - 22s 61ms/step - loss: 0.3644 - accuracy: 0.8817
Epoch 85/150
360/360 [==============================] - 22s 61ms/step - loss: 0.3609 - accuracy: 0.8849
Score for fold  1: loss of 0.38146939873695374; accuracy of 88.42014074325562%
---------------------------------------------------
Training for fold 2 -------
Epoch 1/150
360/360 [==============================] - 24s 57ms/step - loss: 3.4787 - accuracy: 0.0511
Epoch 2/150
360/360 [==============================] - 20s 56ms/step - loss: 3.1014 - accuracy: 0.1149
Epoch 3/150
360/360 [==============================] - 20s 56ms/step - loss: 2.6456 - accuracy: 0.2029
Epoch 4/150
360/360 [==============================] - 20s 56ms/step - loss: 2.2594 - accuracy: 0.2984
Epoch 5/150
3

...

355/355 [==============================] - 27s 76ms/step - loss: 0.5104 - accuracy: 0.8359
Epoch 46/150
355/355 [==============================] - 27s 77ms/step - loss: 0.5043 - accuracy: 0.8380
Epoch 47/150
355/355 [==============================] - 27s 77ms/step - loss: 0.4733 - accuracy: 0.8483
Epoch 48/150
355/355 [==============================] - 27s 76ms/step - loss: 0.4934 - accuracy: 0.8426
Epoch 49/150
355/355 [==============================] - 27s 77ms/step - loss: 0.4837 - accuracy: 0.8451
Epoch 50/150
355/355 [==============================] - 27s 77ms/step - loss: 0.4821 - accuracy: 0.8468
Epoch 51/150
355/355 [==============================] - 27s 77ms/step - loss: 0.4716 - accuracy: 0.8460
Epoch 52/150
355/355 [==============================] - 27s 77ms/step - loss: 0.4718 - accuracy: 0.8486
Epoch 53/150
355/355 [==============================] - 27s 77ms/step - loss: 0.4667 - accuracy: 0.8520
Epoch 54/150
355/355 [==============================] - 27s 76ms/step - loss:

355/355 [==============================] - 21s 60ms/step - loss: 0.4491 - accuracy: 0.8551
Epoch 54/150
355/355 [==============================] - 21s 60ms/step - loss: 0.4519 - accuracy: 0.8528
Epoch 55/150
355/355 [==============================] - 22s 61ms/step - loss: 0.4575 - accuracy: 0.8550
Epoch 56/150
355/355 [==============================] - 21s 61ms/step - loss: 0.4365 - accuracy: 0.8580
Epoch 57/150
355/355 [==============================] - 22s 61ms/step - loss: 0.4407 - accuracy: 0.8579
Epoch 58/150
355/355 [==============================] - 21s 60ms/step - loss: 0.4305 - accuracy: 0.8590
Epoch 59/150
355/355 [==============================] - 22s 61ms/step - loss: 0.4328 - accuracy: 0.8618
Epoch 60/150
355/355 [==============================] - 21s 60ms/step - loss: 0.4252 - accuracy: 0.8642
Epoch 61/150
355/355 [==============================] - 21s 60ms/step - loss: 0.4406 - accuracy: 0.8542
Epoch 62/150
355/355 [==============================] - 22s 61ms/step - loss:

Epoch 18/150
363/363 [==============================] - 34s 94ms/step - loss: 0.8875 - accuracy: 0.7235
Epoch 19/150
363/363 [==============================] - 34s 94ms/step - loss: 0.8568 - accuracy: 0.7283
Epoch 20/150
363/363 [==============================] - 34s 94ms/step - loss: 0.8354 - accuracy: 0.7399
Epoch 21/150
363/363 [==============================] - 34s 95ms/step - loss: 0.8036 - accuracy: 0.7500
Epoch 22/150
363/363 [==============================] - 34s 94ms/step - loss: 0.7750 - accuracy: 0.7582
Epoch 23/150
363/363 [==============================] - 34s 95ms/step - loss: 0.7645 - accuracy: 0.7624
Epoch 24/150
363/363 [==============================] - 34s 95ms/step - loss: 0.7445 - accuracy: 0.7651
Epoch 25/150
363/363 [==============================] - 34s 95ms/step - loss: 0.7325 - accuracy: 0.7729
Epoch 26/150
363/363 [==============================] - 34s 95ms/step - loss: 0.6988 - accuracy: 0.7853
Epoch 27/150
363/363 [==============================] - 34s 95ms

363/363 [==============================] - 28s 77ms/step - loss: 0.7768 - accuracy: 0.7569
Epoch 23/150
363/363 [==============================] - 28s 78ms/step - loss: 0.7719 - accuracy: 0.7530
Epoch 24/150
363/363 [==============================] - 28s 77ms/step - loss: 0.7226 - accuracy: 0.7712
Epoch 25/150
363/363 [==============================] - 28s 78ms/step - loss: 0.7250 - accuracy: 0.7718
Epoch 26/150
363/363 [==============================] - 28s 78ms/step - loss: 0.6893 - accuracy: 0.7880
Epoch 27/150
363/363 [==============================] - 28s 78ms/step - loss: 0.6878 - accuracy: 0.7823
Epoch 28/150
363/363 [==============================] - 28s 78ms/step - loss: 0.6822 - accuracy: 0.7823
Epoch 29/150
363/363 [==============================] - 28s 78ms/step - loss: 0.6771 - accuracy: 0.7922
Epoch 30/150
363/363 [==============================] - 28s 78ms/step - loss: 0.6551 - accuracy: 0.7952
Epoch 31/150
363/363 [==============================] - 28s 78ms/step - loss:

363/363 [==============================] - 23s 63ms/step - loss: 0.6422 - accuracy: 0.8002
Epoch 34/150
363/363 [==============================] - 23s 64ms/step - loss: 0.6144 - accuracy: 0.8111
Epoch 35/150
363/363 [==============================] - 23s 64ms/step - loss: 0.6009 - accuracy: 0.8128
Epoch 36/150
363/363 [==============================] - 23s 64ms/step - loss: 0.5949 - accuracy: 0.8168
Epoch 37/150
363/363 [==============================] - 23s 63ms/step - loss: 0.5730 - accuracy: 0.8194
Epoch 38/150
363/363 [==============================] - 23s 64ms/step - loss: 0.5741 - accuracy: 0.8210
Epoch 39/150
363/363 [==============================] - 23s 64ms/step - loss: 0.5624 - accuracy: 0.8237
Epoch 40/150
363/363 [==============================] - 23s 64ms/step - loss: 0.5768 - accuracy: 0.8206
Epoch 41/150
363/363 [==============================] - 23s 64ms/step - loss: 0.5580 - accuracy: 0.8293
Epoch 42/150
363/363 [==============================] - 23s 64ms/step - loss: